In [1]:
import pandas as pd
import datetime as dt
import requests
import os
from config import config
import yfinance as yf
import numpy as np
import pandas_market_calendars as mcal
from preprocessing.get_and_process_data import *
import yahoo_fin.stock_info as si
import datetime as dt
import pandas as pd

In [2]:
api_key = '26f02b30c6137017402dcd75e2499440'

In [3]:
tic_list = si.tickers_dow()

In [20]:
tic_list[0]

'AAPL'

In [7]:
ls

Data/                                  ress3k_fundamental_final.csv
README.md                              ress3k_fundamental_final_20210901.csv
Russell-3000-Stock-Tickers-List.xlsx   results/
__pycache__/                           run.sh*
code-wrapper.ipynb                     russe3k_21y_stock.csv
config/                                russe3k_21y_stock_with_sector.csv
conm_tic.csv                           russel3k_sector_tic_full.csv
figs/                                  russel3k_tic.csv
fundamental_run_model.py               russle3k_tic_sector_comn.csv
main.py                                sector_tic.csv
models/                                select_top20.py
preprocessing/                         stocks_weight_table20211008.csv
raw-data/                              tables_by_secotr_and_riskLevel/
raw_data/                              tic_cnt.csv
requirements.txt                       top20_risk_sector_stocks.csv


In [4]:
km_file = "raw-data/dow30_km_df.csv"

In [8]:
def get_key_metrics(tic,limit=100):

    api_key = config.api_key
    v3_string = 'https://financialmodelingprep.com/api/v3'
   
    try:
        target_url = f'{v3_string}/key-metrics/{tic}?period=quarter&limit={limit}&apikey={api_key}'
        #print(target_url)
        res = requests.get(target_url).json()
#             #res = res.get('historical')
#             return pd.DataFrame(res).drop(columns = "label")
        return res
    except Exception as e:
        print(e)
        print(f'get_price failed for {tic}')
        pass
    
def get_tics_key_metrics(km_file, tic_list):        
    kms = []
    for tic in tic_list:
        one_km_raw = get_key_metrics(tic,limit=100)
        one_tic_km_raw = pd.DataFrame(one_km_raw)
        kms.append(one_tic_km_raw)
    if len(kms)>0:
        km_df = pd.concat(kms)
    else:
        print("[Error] Encountering error while retrieving key metrics: 0 key metric is retrieved")
        return
    km_df = km_df.drop_duplicates(subset=['date','symbol']).sort_values(['symbol','date']).reset_index(drop=True)
    km_df.to_csv(km_file, index=False)
    return km_df

def get_income_statements(tic,limit=100):
    api_key = config.api_key
    v3_string = 'https://financialmodelingprep.com/api/v3'
    try:
        target_url = f'{v3_string}/income-statement/{tic}?period=quarter&limit={limit}&apikey={api_key}'
        #print(target_url)
        res = requests.get(target_url).json()
        return res
    except Exception as e:
        print(e)
        print(f'get_price failed for {tic}')
        pass 
def get_tics_incom_states(fa_file,tic_list):
    fas = []
    for tic in tic_list:
        one_fa_raw = get_income_statements(tic,limit=100)
        one_tic_fa_raw = pd.DataFrame(one_fa_raw)
        fas.append(one_tic_fa_raw)
    if len(fas)>0:
        fa_df = pd.concat(fas)
    else:
        print("[Error] Encountering error while retrieving income statements: 0 income statement is retrieved")
        return
    fa_df = fa_df.drop_duplicates(subset=['date','symbol']).sort_values(['symbol','date']).reset_index(drop=True)
    fa_df.to_csv(fa_file, index=False)
    return fa_df

def get_stocks_data(stock_file,tic_list,lookback_days):
    def pull_data_by_tic(ticker, start, end):
        df_ticker = yf.download(ticker,start,end, progress=False)
        df_ticker = df_ticker.reset_index()
        df_ticker['tic'] = ticker
        return df_ticker
    end = dt.datetime.now()
    exist_file = os.path.exists(stock_file)
    stocks = []
    for tic in tic_list:
        df_tic = yf.download(tic,  end + dt.timedelta(days=-lookback_days), end, progress=False)
        df_tic = df_tic.reset_index()
        df_tic['tic'] = tic
        stocks.append(df_tic)
    if len(stocks) > 0:
        df = pd.concat(stocks)
        df = df.drop_duplicates(subset=['tic','Date'])
        df.to_csv(stock_file, index=False)
        return df
    print("[Error] Encountering error while retrieving stock data: no stock data are retrieved")
    return None

def processing_raw_data(km_df,fa_df,stocks_df,sector_tic, gvkey_conm_unique, final_file_prefix):

    df1 = km_df[['symbol','date','period','roe','peRatio','priceToSalesRatio','evToSales','pbRatio',
                        'pocfratio','currentRatio','enterpriseValueOverEBITDA','evToOperatingCashFlow',
                        'debtToAssets','netDebtToEBITDA','debtToEquity','payablesTurnover','daysSalesOutstanding',
                        'daysPayablesOutstanding']]
    df1 = df1.rename(columns = {'symbol':'tic','period':'quarter','roe':'X4_ROE','peRatio':'X5_PE',
                                'priceToSalesRatio':'X6_PS','evToSales':'X9_OM','pbRatio':'X10_PB','pocfratio':'X11_PCFO',
                                'currentRatio':'X12_CR','enterpriseValueOverEBITDA':'X13_EM','evToOperatingCashFlow':'X14_EVCFO',
                                'debtToAssets':'X15_LTDTA','netDebtToEBITDA':'X16_WCR','debtToEquity':'X17_DE','payablesTurnover':'X18_QR',
                                'daysSalesOutstanding':'X19_DSI','daysPayablesOutstanding':'X20_DPO'})
    df1['datadate'] = pd.to_datetime(df1['date'])
    df1['datadate'] = df1['datadate'].apply(lambda x: dt.datetime.strftime(x,'%Y%m%d'))
    df1 = df1.set_index(['datadate','tic'])
    
    
    df2 = fa_df[['date','symbol','period','revenue','eps','operatingIncomeRatio',
                            'netIncomeRatio','grossProfitRatio']]
    df2 = df2.rename(columns={'symbol':'tic','revenue':'X1_REVGH','eps':'X2_EPS','operatingIncomeRatio':'X3_ROA',
                              'netIncomeRatio':'X7_NPM','grossProfitRatio':'X8_GPM'})
    df2['datadate2'] = pd.to_datetime(df2['date'])
    df2['datadate'] = df2['datadate2'].apply(lambda x: dt.datetime.strftime(x,'%Y%m%d'))
    df2 = df2.set_index(['datadate','tic'])                                                

    df = pd.merge(df1,df2.drop('date',axis=1), on=(['datadate','tic']), how='inner')
    
    df = df.reset_index()
    df['fyearq'] = df['datadate2'].apply(lambda x: dt.datetime.strftime(x,'%Y'))
    df['datacqtr'] = df['datadate2'].apply(get_quarters)
    df['datafqtr'] = df['fyearq']+ df['quarter']
    df['tradedate'] = df['datadate2'].apply(get_trade_date)
    df['fqtr'] = df['datadate2'].apply(get_quarter_num)                                                
    
    
    df_gvkey = df.join(gvkey_conm_unique, on='tic',how='left')
                                                     
    fakm_final = df_gvkey[['tic','datadate','tradedate','fyearq','fqtr','conm','datacqtr','datafqtr',
                         'X1_REVGH','X2_EPS','X3_ROA','X4_ROE','X5_PE','X6_PS','X7_NPM','X8_GPM','X9_OM','X10_PB',
                         'X11_PCFO','X12_CR','X13_EM','X14_EVCFO','X15_LTDTA','X16_WCR','X17_DE','X18_QR','X19_DSI',
                         'X20_DPO']] 
    fc = ['X1_REVGH','X2_EPS','X3_ROA','X4_ROE','X5_PE','X6_PS','X7_NPM','X8_GPM','X9_OM','X10_PB',
                         'X11_PCFO','X12_CR','X13_EM','X14_EVCFO','X15_LTDTA','X16_WCR','X17_DE','X18_QR','X19_DSI',
                         'X20_DPO']
    fakm_final.to_csv('fakm_final.csv')
    fakm_final['month'] = fakm_final['tradedate'].astype(int)//100
 
    # combine with y_return from stock_df   

    stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
    stocks_df['datadate'] = pd.to_datetime(stocks_df['Date']).apply(lambda x: dt.datetime.strftime(x,'%Y%m%d'))
    stocks_df['datadate'] = stocks_df['datadate'].astype(int)

    nyse = mcal.get_calendar('NYSE')
    trade_dates = nyse.schedule(start_date=stocks_df['Date'].min()+dt.timedelta(days=-60), end_date=stocks_df['Date'].max()+dt.timedelta(days=210))
    trade_dates_list = list(trade_dates.index)
    
    trading_dates = []
    for i in range(len(trade_dates_list)):
        date = trade_dates_list[i].strftime('%Y%m%d')
        trading_dates.append(int(date))
    
    #trading_dates_set = set(trading_dates)
    tdf = pd.DataFrame({'trading_date':trading_dates})
    tdf['month'] = tdf['trading_date'].astype(int)//100
    
    tdfm = tdf.groupby('month')['trading_date'].min().reset_index()

    funda_df = fakm_final.join(tdfm.set_index('month'),on='month',how='left')
    funda_df = funda_df.rename(columns={'tradedate':'tradedate_fake','trading_date':'tradedate'})
    print(funda_df['tradedate'].shape, funda_df[['tradedate']].dropna().shape)
    print(funda_df[funda_df['tradedate'].isna()])
    #funda_df['tradedate'] = funda_df['tradedate'].astype(int)
    stocks_df['tradedate'] = stocks_df['datadate'].astype(int)
    stocks_df['tradedate_price'] = stocks_df['Adj Close']
    
    fa_quarter_prices = funda_df.join(stocks_df[['tradedate','tradedate_price','tic']].set_index(['tradedate','tic']),
                                      on=['tradedate','tic'],how='left').dropna()

    fa_quarter_prices['next_trading_month'] = fa_quarter_prices['tradedate'].apply(lambda x: dt.datetime.strptime(str(int(x)), '%Y%m%d')).apply(get_trade_month)
    fa_prices_df = fa_quarter_prices.join(tdfm.rename(columns={'month':'next_trading_month',
                                                                'trading_date':'next_trading_date'}).set_index('next_trading_month'),
                                                                  on='next_trading_month',how='left')

    fa_prices_df['next_trading_date'] = fa_prices_df['next_trading_date'].apply(lambda x: int(x) if x and ~np.isnan(x) else None)
    
    stocks_df['next_trading_date'] = stocks_df['datadate'].astype(int)
    stocks_df['next_trading_date_price'] = stocks_df['Adj Close']
    all_quartly_prices = fa_prices_df.join(stocks_df[['next_trading_date','next_trading_date_price','tic']].set_index(['next_trading_date','tic']),
                                      on=['next_trading_date','tic'],how='left')
    all_quartly_prices['y_return'] = np.log(all_quartly_prices['next_trading_date_price']/all_quartly_prices['tradedate_price'])

    
    fundamental_all = pd.merge(all_quartly_prices,sector_tic[['GICS Sector','tic']], on='tic',how='left')
    sector_list = fundamental_all['GICS Sector'].unique().tolist()
    
    sector_map = {}
    for i in range(len(sector_list)):
        sector_map[sector_list[i]] = i*5+10
        
    fundamental_all['GICS Sector'] = fundamental_all['GICS Sector'].astype(str)
    fundamental_all['gsector'] = fundamental_all['GICS Sector'].apply(lambda x: -1 if x not in sector_map else sector_map[x])
    #del sp500_fundamental_final['GICS Sector']
    print(fundamental_all.tail())
    
    fundamental_clean_table = fundamental_all
    max_trade_date = fundamental_clean_table.tradedate.max()
    keep = fundamental_clean_table[(fundamental_clean_table.tradedate == max_trade_date)]
    keep = keep.append(fundamental_clean_table[(fundamental_clean_table.tradedate < max_trade_date)].dropna(subset=['y_return']+fc))
    keep['tradedate'] = keep['tradedate'].astype(int)
    max_trade_date = keep.tradedate.max()
    final_file = final_file_prefix + f'_{max_trade_date}.csv'
    keep.to_csv(final_file,index=False)
    keep.to_csv(final_file_prefix+'.csv',index=False)
    return keep     

In [9]:
industry_indices = ['dow30','sp500']
idx = 'dow30'
km_file, fa_file, stock_file = f"raw-data/{idx}_km_df.csv", f"raw-data/{idx}_fa_df.csv", f"raw-data/{idx}_stock_df.csv"

In [10]:
km_df = get_tics_key_metrics(km_file,tic_list)


In [12]:
fa_df = get_tics_incom_states(fa_file,tic_list)

In [13]:
stock_df = get_stocks_data(stock_file,tic_list,lookback_days=150)

In [82]:
pwd

'/Users/tianlong/Downloads/Dynamic-Stock-Recommendation_final_folder'

In [16]:
sector_tic = pd.read_csv('sector_tic.csv')
gvkey_conm = pd.read_csv('conm_tic.csv')

In [27]:
gvkey_conm['conm'] = gvkey_conm['conm'].astype(str)
gvkey_conm['tic'] = gvkey_conm['tic'].astype(str)
gvkey_conm.tic

0          A
1         AA
2       AAIC
3        AAL
4        AAN
        ... 
2954     ZTS
2955    ZUMZ
2956     ZUO
2957    ZYXI
2958    TRUE
Name: tic, Length: 2959, dtype: object

In [31]:
for i, row in gvkey_conm.iterrows():
    if not type(row.tic) == 'str' or not :
        type(row.tic)

In [24]:
fundamental = processing_raw_data(km_df,fa_df,stock_df,sector_tic, gvkey_conm,'russell3k')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat